Task: implement a spreadsheet engine that had an interface with two methods set_cell and get_cell

In [17]:
# version 1: resolve on read
class Sheet:
    def __init__(self):
        # key -> value OR [key]
        self.cells = {}

    # resolve on get
    def get_cell(self, key):
        # if cell is not in the dictionary, return 0
        if key not in self.cells:
            return 0
        # else if cell is a list of cells, return the sum of the cells
        elif type(self.cells[key]) == list:
            return sum([self.get_cell(k) for k in self.cells[key]])
        # else return the value of the cell
        else:
            return self.cells[key]

    # O(1) set
    def set_cell(self, key, value):
        self.cells[key] = value

In [180]:
# version 2: resolve on write
class Sheet2:
    def __init__(self):        
        # which cells does this cell depend on?
        self.deps = {} # key -> [key]
        
        # which cells depend on this cell?
        self.index = {} # key -> {key}

        # what is the value of this cell?
        self.values = {} # key -> value
        
    # O(1) get
    def get_cell(self, key):
        if key not in self.values:
            return 0
        else:
            return self.values[key]
    
    # resolve on set
    def set_cell(self, key, value):
        # clear the index
        if key in self.deps:
            for v in self.deps[key]:
                self.index[v].remove(key)
        # clear the deps 
        if key in self.deps:
            del self.deps[key] 
        # clear the value
        if key in self.values:
            del self.values[key]
        
        # if value is a list of cells...  
        if type(value) == list:
            # set them as deps for this cell
            self.deps[key] = value
            # add this cell's key to the index for each dep cell
            for v in value:
                if v in self.index:
                    self.index[v].add(key)
                else:
                    self.index[v] = {key}
            # resolve the value
            self._resolve(key)
        # else just set the value 
        else:
            self.values[key] = value
            # if this cell is a dep for other cells, resolve them
            if key in self.index:
                for v in self.index[key]:
                    self._resolve(v)

    # resolve the value of a cell and all cells that depend on it
    def _resolve(self, key): 
        self.values[key] = sum([self.values[k] for k in self.deps[key]])
        if key in self.index:
            for v in self.index[key]:
                self._resolve(v)
        

In [181]:
sheet = Sheet2()

# A cells value should update if its dependents change
sheet.set_cell("A1", 1)
sheet.set_cell("A2", 2)
sheet.set_cell("A3", ["A1", "A2"])
sheet.set_cell("A1", 3)
assert sheet.get_cell("A3") == 5, "A3 should be 5"

# A cell should be able to depend on another directly and indirectly
sheet.set_cell("A4", ["A3", "A1"]);
assert sheet.get_cell("A4") == 8, "A4 should be 8"

# A cell should be able to depend on another cell multiple times
sheet.set_cell("A5", ["A1", "A1"]);
assert sheet.get_cell("A5") == 6, "A5 should be 6"

# A cell should be able to only depend on other non-leaf cells
sheet.set_cell("A6", ["A5", "A4"]);
assert sheet.get_cell("A6") == 14, "A6 should be 14"

# Updating a cell should update the values for all cells that depend on it, directly and indirectly
sheet.set_cell("A1", 1);
assert sheet.get_cell("A3") == 3, "A3 should be 3"
assert sheet.get_cell("A4") == 4, "A4 should be 4"
assert sheet.get_cell("A5") == 2, "A5 should be 2"
assert sheet.get_cell("A6") == 6, f"A6 should be 6 but got {sheet.get_cell('A6')}"

# A cell should be able to depend on more than 2 cells
sheet.set_cell("A7", ["A1", "A1", "A2"]);
assert sheet.get_cell("A7") == 4, "A7 should be 1 + 1 + 2"
sheet.set_cell("A2", 3);
assert sheet.get_cell("A7") == 5, "A7 should be 1 + 1 + 3"

In [182]:
sheet.deps

{'A3': ['A1', 'A2'],
 'A4': ['A3', 'A1'],
 'A5': ['A1', 'A1'],
 'A6': ['A5', 'A4'],
 'A7': ['A1', 'A1', 'A2']}

In [183]:
sheet.index

{'A1': {'A3', 'A4', 'A5', 'A7'},
 'A2': {'A3', 'A7'},
 'A3': {'A4'},
 'A5': {'A6'},
 'A4': {'A6'}}

In [184]:
sheet.values

{'A3': 4, 'A4': 5, 'A5': 2, 'A6': 7, 'A1': 1, 'A7': 5, 'A2': 3}